In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0, "../")

In [21]:
from nohomers.notebook.widgets import expand2square, image_labeller_widget, display_df_images
from nohomers.ml.torch_datasets import SimpleVisionExample, SimpleVisionDataset, split_train_valid_test, pil_loader
from nohomers.ml.lightweight_gan_trainer import get_trainer, generate_images
from nohomers.ml.cleaner import make_train_test_datasets, train_cleaner, make_network_input_from_images, load_cleaner, save_cleaner
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
import pandas as pd
import numpy as np
import torch
import torchvision
import pydash as py_
from tqdm.auto import tqdm
from uuid import uuid4
from PIL import Image
from pathlib import Path
import json
import copy
import shutil
from multiprocessing.pool import ThreadPool
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from collections import defaultdict, Counter
import re

In [3]:
base_folder = Path("/mnt/evo/projects/nohomers/cleaners/selection")

In [4]:
## If you want to remove
try:
    shutil.rmtree(base_folder)
except FileNotFoundError:
    pass
base_folder.mkdir(parents=True, exist_ok=True)

In [6]:
checkpoints = {25, 30, 35, 40, 45, 50}

In [7]:
for checkpoint in checkpoints:
    trainer = get_trainer(
        models_dir="/mnt/evo/projects/metapedia/tmp/stylegan2/models", 
        name="simpsons_large_cleaned_nobackground_1024_augall03_sle_res64",
        load_from=checkpoint,
    )
    
    num_examples = 500
    per_batch = 100
    with ThreadPool(32) as pool:
        for batch in tqdm(range(0, num_examples, per_batch)):
            images = generate_images(trainer, per_batch)
            list(
                pool.imap(lambda x: x.image.save(base_folder / f"checkpoint_{checkpoint}_{uuid4()}.png"), images)
            )

loading from version 0.12.4



loading from version 0.12.4



loading from version 0.12.4



loading from version 0.12.4



loading from version 0.12.4



loading from version 0.12.4


In [8]:
images = list(base_folder.glob("*.png"))
labels = {}

In [27]:

counts = defaultdict(Counter)
for path, label in labels.items():
    m = re.match(r"checkpoint_(\d+)_.*", path.name)
    checkpoint = m.group(1)
    counts[checkpoint][label] += 1 

In [28]:
for checkpoint, counts in sorted(counts.items()):
    amazing, good, bad = [counts[e] for e in ("Amazing", "Good", "Bad")]
    passing = amazing + good
    total_counts = amazing + good + bad
    print(f"{checkpoint}\tAmazing {amazing / total_counts:.2f}\tGood {good / total_counts:.2f}\tBad {bad / total_counts:.2f}\t Pass={passing / total_counts:.2f}")

25	Amazing 0.02	Good 0.35	Bad 0.62	 Pass=0.38
30	Amazing 0.03	Good 0.33	Bad 0.63	 Pass=0.37
35	Amazing 0.06	Good 0.44	Bad 0.50	 Pass=0.50
40	Amazing 0.08	Good 0.36	Bad 0.56	 Pass=0.44
45	Amazing 0.07	Good 0.32	Bad 0.61	 Pass=0.39
50	Amazing 0.06	Good 0.23	Bad 0.71	 Pass=0.29


In [16]:
to_label = py_.sample_size(images, 100)
image_labeller_widget(to_label, labels, ["N/A", "Amazing", "Good", "Bad"], image_size=(250, 250))